In [ ]:
import os
from pathlib import Path

try:
    import google.colab

    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import userdata

    repo_name = "dgcnz/dl2"
    url = f"https://{userdata.get('gh_pat')}@github.com/{repo_name}.git"
    !git clone {url}
    print("\nCurrent Directory:")
    %cd dl2
    !pip install lightning
    #!pip install -r requirements.txt

else:  # automatically checks if the current directory is 'repo name'
    curdir = Path.cwd()
    print("Current Directory", curdir)
    assert (
        curdir.name == "dl2" or curdir.parent.name == "dl2"
    ), "Notebook cwd has to be on the project root"
    if curdir.name == "notebooks":
        %cd ..
        print("New Current Directory:", curdir)

In [ ]:
import sys

sys.path.append(".")

In [ ]:
import numpy as np
import torch
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms import (
    Compose,
    InterpolationMode,
    Pad,
    RandomRotation,
    Resize,
    ToTensor,
)

# these imports always confuse me
from src.data.rotated_mnist_datamodule import MnistRotDataset

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# simple test
raw_mnist_test = MnistRotDataset(root="data/", train=False, download=True)
# retrieve the first image from the test set
x, y = next(iter(raw_mnist_test))
print(x)
print(y)

In [ ]:
# images are padded to have shape 29x29.
# this allows to use odd-size filters with stride 2 when downsampling a feature map in the model
pad = Pad((0, 0, 1, 1), fill=0)

# to reduce interpolation artifacts (e.g. when testing the model on rotated images),
# we upsample an image by a factor of 3, rotate it and finally downsample it again
resize1 = Resize(87)
resize2 = Resize(29)

totensor = ToTensor()

train_transform = Compose(
    [
        pad,
        resize1,
        RandomRotation(180.0, interpolation=InterpolationMode.BILINEAR, expand=False),
        resize2,
        totensor,
    ]
)

mnist_train = MnistRotDataset(root="data/", train=True, transform=train_transform)
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=64)


test_transform = Compose(
    [
        pad,
        totensor,
    ]
)
mnist_test = MnistRotDataset(root="data/", train=True, transform=test_transform)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=64)